In [1]:
## Trained model for story generator - using the gpt_2_simple
## Forked the original github repo to my repo - https://github.com/rbewoor/gpt-2-simple

## Anaconda environment setup as follows:
#	*** create new environment
#conda create -n --newENV python=3.7
#
#	*** general stuff
#conda install jupyter
#
#	*** Story Generation
#conda install numpy
#conda install requests
#conda install regex
#pip install tensorflow==1.15.2
#** not done this so far pip install toposort
#pip install gpt-2-simple			## Successfully installed gpt-2-simple-0.7.1 regex-2020.10.23 toposort-1.5

In [2]:
import os
import json

import gpt_2_simple as gpt2
import tensorflow as tf

from gpt_2_simple.src import model, sample, encoder, memory_saving_gradients
from gpt_2_simple.src.load_dataset import load_dataset, Sampler
from gpt_2_simple.src.accumulate import AccumulatingOptimizer

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
assert tf.__version__ < '2.0.0', "gpt-2-simple currently does not support " \
    "TensorFlow 2.0. You'll need to use a virtualenv/cloud computer which " \
    "has Tensorflow 1.X on it."

In [4]:
DEBUG_SWITCH = False

gui_img_cap_show_results_logic_RC = 0
gui_img_cap_module_results = \
[
    {
        'key_elements': ['bicycle', 'person'],
        'selected_images': [
            ['/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Data/COCO_test2017_41k/test2017/000000155758.jpg', 'man riding bike in a park']
            ]
            },
    {
        'key_elements': ['tvmonitor', 'person'],
        'selected_images': [
            ['/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Data/COCO_test2017_41k/test2017/000000287296.jpg', 'young girl standing in front of tv playing video game'],
            ['/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Data/COCO_test2017_41k/test2017/000000175337.jpg', 'man walking past a tv in his house']
            ]
            },
    {
        'key_elements': ['handbag'],
        'selected_images': [
            ['/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Data/COCO_test2017_41k/test2017/000000117100.jpg', 'two suitcases are in the boot of a car while a woman holds a hand bag in her hand'],
            ['/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/Data/COCO_test2017_41k/test2017/000000313060.jpg', 'woman is holding her hang bag while speaking on her cell phone']
            ]
            }
]

In [5]:
###############################################################################################################
## -------------------     STORY GENERATOR STARTS                  STORY GENERATOR STARTS   -------------------
###############################################################################################################
def my_load_gpt2(_sess,
              _checkpoint='latest',
              _run_name="run1",
              _checkpoint_dir="checkpoint",
              _model_name=None,
              _model_dir='models',
              _multi_gpu=False):
    """Loads the model checkpoint or existing model into a TensorFlow session for repeated predictions.
    """

    if _model_name:
        checkpoint_path = os.path.join(_model_dir, _model_name)
    else:
        checkpoint_path = os.path.join(_checkpoint_dir, _run_name)
    print(f"\ncheckpoint_path = {checkpoint_path}\n")

    hparams = model.default_hparams()
    with open(os.path.join(checkpoint_path, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    context = tf.compat.v1.placeholder(tf.int32, [1, None])

    gpus = []
    if _multi_gpu:
        gpus = get_available_gpus()

    output = model.model(hparams=hparams, X=context, gpus=gpus)

    if _checkpoint=='latest':
        ckpt = tf.train.latest_checkpoint(checkpoint_path)
    else:
        ckpt = os.path.join(checkpoint_path,_checkpoint)

    saver = tf.compat.v1.train.Saver(allow_empty=True)
    _sess.run(tf.compat.v1.global_variables_initializer())

    if _model_name:
        print(f"\nLoading pretrained model :: {ckpt}\n")
    else:
        print(f"\nLoading checkpoint :: {ckpt}\n")
    saver.restore(_sess, ckpt)

def reset_session(_sess, threads=-1, server=None):
    """Resets the current TensorFlow session, to clear memory
    or load another model.
    """

    tf.compat.v1.reset_default_graph()
    _sess.close()

def sty_gen_inference_functionality(_gui_img_cap_module_results, _sg_params, _DEBUG_SWITCH):
    _seed_for_styGen, SG_LENGTH, SG_TEMPERATURE, SG_NSAMPLES, SG_BATCH_SIZE, RUN_NAME = _sg_params
    
    sty_gen_inference_module_results = []
    
    
    ## set up the pre-trained GPT-2 model from checkpoint directory
    
    ## copy checkpoint directory from gdrive - not required as saved it locally
    ##gpt2.copy_checkpoint_from_gdrive(run_name='run1')
    GPT2_CHECKPOINT_DIR = r'/home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/'
    #RUN_NAME = r"run1"

    sess = gpt2.start_tf_sess()
    print(f"\nTF session started\n")
    #gpt2.load_gpt2(sess, run_name='run1')
    my_load_gpt2(
        _sess = sess,
        _checkpoint='latest',
        _run_name=RUN_NAME,
        _checkpoint_dir=GPT2_CHECKPOINT_DIR,
        _model_name=None,
        _model_dir='models',
        _multi_gpu=False)
    
    print(f"\n\nGenerating the story using 'prefix' = {_seed_for_styGen}\n")
    gpt2.generate(sess,
                  length=SG_LENGTH,
                  temperature=SG_TEMPERATURE,
                  prefix=seed_for_styGen,
                  nsamples=SG_NSAMPLES,
                  batch_size=SG_BATCH_SIZE,
                  checkpoint_dir=GPT2_CHECKPOINT_DIR
                 )
    
    reset_session(sess)
    print(f"\n\nFINISHED generating the story\n")

    return (0, None, sty_gen_inference_module_results)

###############################################################################################################
## ---------------------     STORY GENERATOR ENDS                  STORY GENERATOR ENDS   ---------------------
###############################################################################################################

In [8]:
## Parameters to generate the story
## NOTE: nsamples % batch_size == 0
SG_LENGTH = 300
SG_TEMPERATURE = 0.95
SG_NSAMPLES = 1
SG_BATCH_SIZE = 1
RUN_NAME = r"Run2_File11_2_checkpoint_run2"

## seed value
img_cap = r'A man is cycling down the road'
seed_for_styGen = img_cap.capitalize() + r' '

## check samples and batch size values are compatible
assert (SG_NSAMPLES % SG_BATCH_SIZE == 0) , f"Values for NSAMPLES and BATCH_SIZE incompatible. " \
        f"NSAMPLES={SG_NSAMPLES} % SG_BATCH_SIZE={SG_BATCH_SIZE} != 0"

SG_PARAMS = (seed_for_styGen, SG_LENGTH, SG_TEMPERATURE, SG_NSAMPLES, SG_BATCH_SIZE, RUN_NAME)

sty_gen_inference_logic_RC, sty_gen_inference_logic_msg, sty_gen_inference_module_results = sty_gen_inference_functionality(gui_img_cap_module_results, SG_PARAMS, DEBUG_SWITCH)


TF session started


checkpoint_path = /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2


Loading checkpoint :: /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000

INFO:tensorflow:Restoring parameters from /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000


Generating the story using 'prefix' = A man is cycling down the road 

A man is cycling down the road !"
he said.
The only thing he received from one of his peasant helper in the village was a jibe about his back ; and he could not get the reward of it, even upon his friend.
In some ways Thuger had succeeded ; and in others he had failed ; but every year something happened to him, and came over the pulpit.
The idea of a man going naked in battle was not at all new, and there were times when he was much struck by the terrible unreality of it all.
The Christian fanatic never rode or learned ; every day he ailed his boughs to Ma

In [7]:
## Parameters to generate the story
## NOTE: nsamples % batch_size == 0
SG_LENGTH = 300
SG_TEMPERATURE = 0.7
SG_NSAMPLES = 1
SG_BATCH_SIZE = 1
RUN_NAME = r"Run2_File11_2_checkpoint_run2"

## seed value
img_cap = r'A man is cycling down the road'
seed_for_styGen = img_cap.capitalize() + r' '

## check samples and batch size values are compatible
assert (SG_NSAMPLES % SG_BATCH_SIZE == 0) , f"Values for NSAMPLES and BATCH_SIZE incompatible. " \
        f"NSAMPLES={SG_NSAMPLES} % SG_BATCH_SIZE={SG_BATCH_SIZE} != 0"

SG_PARAMS = (seed_for_styGen, SG_LENGTH, SG_TEMPERATURE, SG_NSAMPLES, SG_BATCH_SIZE, RUN_NAME)

sty_gen_inference_logic_RC, sty_gen_inference_logic_msg, sty_gen_inference_module_results = sty_gen_inference_functionality(gui_img_cap_module_results, SG_PARAMS, DEBUG_SWITCH)


TF session started


checkpoint_path = /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2


Loading checkpoint :: /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000

INFO:tensorflow:Restoring parameters from /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000


Generating the story using 'prefix' = A man is cycling down the road 

A man is cycling down the road ich a fine, handsome man, riding a horse 's head, and looking at me with his eyes full . ''
`` I will give nothing, '' said the young man, `` to take this man ; I will not even leave my horse.
I am sure I am going to hell, '' he added, turning and turning his head.
The boy glanced at his master and nodded.
`` The boy is a fool, '' he muttered to himself.
`` He thinks he is going to hell, and I do not care a bit what happens.
If he comes to me at night, I will kill him.
It is good to have one 's head shot.
But I will have him 

In [7]:
## Parameters to generate the story
## NOTE: nsamples % batch_size == 0
SG_LENGTH = 300
SG_TEMPERATURE = 0.7
SG_NSAMPLES = 3
SG_BATCH_SIZE = 3
RUN_NAME = r"Run2_File11_2_checkpoint_run2"

## seed value
img_cap = r'A man is cycling down the road'
seed_for_styGen = img_cap.capitalize() + r' '

## check samples and batch size values are compatible
assert (SG_NSAMPLES % SG_BATCH_SIZE == 0) , f"Values for NSAMPLES and BATCH_SIZE incompatible. " \
        f"NSAMPLES={SG_NSAMPLES} % SG_BATCH_SIZE={SG_BATCH_SIZE} != 0"

SG_PARAMS = (seed_for_styGen, SG_LENGTH, SG_TEMPERATURE, SG_NSAMPLES, SG_BATCH_SIZE, RUN_NAME)

sty_gen_inference_logic_RC, sty_gen_inference_logic_msg, sty_gen_inference_module_results = sty_gen_inference_functionality(gui_img_cap_module_results, SG_PARAMS, DEBUG_SWITCH)


TF session started


checkpoint_path = /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2


Loading checkpoint :: /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000

INFO:tensorflow:Restoring parameters from /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000


Generating the story using 'prefix' = A man is cycling down the road 

A man is cycling down the road urns, '' he explained, `` and he is banging his head against the wall.
He is not strong enough to carry out a big man like the Sentinel, but I am sure he is strong enough for him.
He is going to be here this afternoon, and I shall not be here till he comes . ''
`` You must go, '' I retorted.
I saw the Sentinel, too.
I knew his face, and I knew that I had killed him.
He had been in the meantime.
I was far as I could get, and I did not believe I had got to go till it was all over.
I had been cutting my way through the trees, an

In [6]:
## Parameters to generate the story
## NOTE: nsamples % batch_size == 0
SG_LENGTH = 300
SG_TEMPERATURE = 0.7
SG_NSAMPLES = 3
SG_BATCH_SIZE = 3
RUN_NAME = r"Run2_File11_2_checkpoint_run2"

## seed value
img_cap = r'A man is cycling down the road. A woman is carrying a hand bag. A boy is watching televsion.'
seed_for_styGen = img_cap.capitalize() + r' '

## check samples and batch size values are compatible
assert (SG_NSAMPLES % SG_BATCH_SIZE == 0) , f"Values for NSAMPLES and BATCH_SIZE incompatible. " \
        f"NSAMPLES={SG_NSAMPLES} % SG_BATCH_SIZE={SG_BATCH_SIZE} != 0"

SG_PARAMS = (seed_for_styGen, SG_LENGTH, SG_TEMPERATURE, SG_NSAMPLES, SG_BATCH_SIZE, RUN_NAME)

sty_gen_inference_logic_RC, sty_gen_inference_logic_msg, sty_gen_inference_module_results = sty_gen_inference_functionality(gui_img_cap_module_results, SG_PARAMS, DEBUG_SWITCH)


TF session started


checkpoint_path = /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2


Loading checkpoint :: /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000

INFO:tensorflow:Restoring parameters from /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000


Generating the story using 'prefix' = A man is cycling down the road. a woman is carrying a hand bag. a boy is watching televsion. 

A man is cycling down the road. a woman is carrying a hand bag. a boy is watching televsion. ���
`` It is an old woman, '' said Ben, who was not about to start.
`` She is a homeless woman, and he is a man.
It is the poor old woman that has hurt him . ''
`` She is not a poor woman, '' said Mrs. Spring, looking at the man who came in.
`` She has a little head of red hair.
It is not a hair, but it is a hair.
She is a poor woman.
Poor little man!
I am afraid he will be a man to you.
I am afraid he h